# Advent of Code 2020

In [ ]:
import numpy as np
import pandas as pd
import itertools
import re
import functools
import awkward as ak
import numba as nb

## Day 1
### part 1

In [ ]:
with open('1.txt') as numberfile:
    numbers = np.array(numberfile.read().split(), dtype=int)

In [ ]:
for i in numbers:
    if (product := numbers[numbers + i == 2020] * i).size > 0:
        print(product)

### part 2

In [ ]:
for i, n in enumerate(numbers):
    for m in numbers[i:]:
        if (product := numbers[numbers + n + m == 2020] * n * m).size > 0:
            print(product)

## Day 2
### part 1

In [ ]:
passwords = pd.read_csv('2.txt', sep=' |-|: ', 
                        names=['min_counts', 'max_counts', 'character', 'password'], 
                        engine='python')

In [ ]:
valid_passwords = 0
for entry in passwords.itertuples():
    if entry.min_counts <= entry.password.count(entry.character) <= entry.max_counts:
        valid_passwords += 1
print(valid_passwords)

### part 2

In [ ]:
passwords.rename(columns={'min_counts': 'first_index', 'max_counts': 'last_index'}, inplace=True)
valid_passwords = 0
for entry in passwords.itertuples():
    if (entry.password[entry.first_index-1] == entry.character) ^ (entry.password[entry.last_index-1] == entry.character):
        valid_passwords += 1
print(valid_passwords)

## Day 3
### part 1

In [ ]:
tree_counter = 0
index = 0
slope = 3
with open('3.txt') as tree_pattern_file:
    for row in tree_pattern_file:
        row = row[:-1]
        tree_counter += (row[index] == '#')
        index = (index + slope) % (len(row))
print(tree_counter)

### part 2

In [ ]:
slope = np.arange(1, 8, 2, dtype='int64')
index = np.zeros_like(slope)
tree_counter = np.zeros_like(slope)

with open('3.txt') as tree_pattern_file:
    for row in tree_pattern_file:
        row = row[:-1]
        tree_counter += [row[i] == '#' for i in index]
        index = (index + slope) % (len(row))

# Special case
s_slope = 1
s_index = 0
s_tree_counter = 0
with open('3.txt') as tree_pattern_file:
    for row in itertools.islice(tree_pattern_file, 0, None, 2):
        row = row[:-1]
        s_tree_counter += (row[s_index] == '#')
        s_index = (s_index + s_slope) % (len(row))
print(tree_counter.prod() * s_tree_counter)

## Day 4
### part 1

In [ ]:
with open('4.txt') as passport_file:
    passports = passport_file.read()
pass_port_keys = ['byr','iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
valid_passports = 0
for p_info in passports.split('\n\n'):
    valid_passports += all(key in p_info for key in pass_port_keys)
print(valid_passports)

### part 2

In [ ]:
def create_dict(passport_info):
    for info in passport_info.split():
        yield info.split(':')

In [ ]:
with open('4.txt') as passport_file:
    passports = passport_file.read()
passport_keys = ['byr','iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid', 'cid']
valid_passports = 0
for p_info in passports.split('\n\n'):
    p_info_dict = {key: value for key, value in create_dict(p_info)}
    # Guard clauses for input validation
    try:
        if not (set(p_info_dict.keys()) | set(('cid',))) == set(passport_keys):
            continue
        elif not 1920 <= int(p_info_dict['byr']) <= 2002:
            continue
        elif not 2010 <= int(p_info_dict['iyr']) <= 2020:
            continue
        elif not 2020 <= int(p_info_dict['eyr']) <= 2030:
            continue
        elif not ((('cm' in p_info_dict['hgt']) and (150 <= int(p_info_dict['hgt'][:-2]) <= 193))
                  or ('in' in p_info_dict['hgt'] and (59 <= int(p_info_dict['hgt'][:-2]) <= 76))):
            continue
        elif re.fullmatch('#(?:[0-9a-f]{6})', p_info_dict['hcl']) is None:
            continue
        elif p_info_dict['ecl'] not in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
            continue
        elif not (p_info_dict['pid'].isnumeric() and len(p_info_dict['pid']) == 9):
            continue
    except Exception as e:
        print(p_info_dict, e)
    valid_passports += 1
    
print(valid_passports)

## Day 5
### part 1

In [ ]:
a = 'i'
a.replace('i', '0')
a

In [ ]:
with open('5.txt') as seat_file:
    seats = seat_file.read()
bin_seats = seats.replace('F', '0')
bin_seats = bin_seats.replace('B', '1')
bin_seats = bin_seats.replace('L', '0')
bin_seats = bin_seats.replace('R', '1')
rows = np.array([int(s[:-3], base=2) for s in bin_seats.split('\n')], dtype=int)
columns = np.array([int(s[-3:], base=2) for s in bin_seats.split('\n')], dtype=int)
seat_id = np.array([int(s, base=2) for s in bin_seats.split('\n')])

print((rows*8+columns).max(), seat_id.max()) # Both should produce same output

### part 2

In [ ]:
seat_id.sort()
seat_id[np.append(np.diff(seat_id) != 1, False)] + 1

## Day 6
### part 1

In [ ]:
sum(len(set(group_answers.replace('\n', ''))) for group_answers in open('6.txt').read().split('\n\n'))

In [ ]:
sum(len(functools.reduce(set.__and__, (set(individual_answer) for individual_answer in group.split('\n'))))
    for group in open('6.txt').read().split('\n\n'))

## Day 7
### part 1

In [ ]:
def count_outer_bags(bag_name):
    outer_bag_index = bags['inner'].str.contains(bag_name)
    if outer_bag_index.sum() == 0:
        return set()
    return set(bags['outer'][outer_bag_index]) | functools.reduce(set.__or__, (count_outer_bags(bag) for bag in bags['outer'][outer_bag_index]))

In [ ]:
bags = pd.read_csv('7.txt', sep='bags contain ', names=['outer', 'inner'], engine='python')
len(count_outer_bags('shiny gold'))

### part 2

In [ ]:
bag_rules = {}
with open('7.txt') as bag_rule_file:
    for bag_rule in bag_rule_file:
        bag_rule = bag_rule.strip('\n')
        bag_name = bag_rule[:bag_rule.index(' bags contain ')]
        if bag_rule.endswith('contain no other bags.'):
            bag_rules[bag_name] = []
        else:
            # I barely know what this line does but it works
            inner_bags = re.split(' bags.| bags, | bag.| bag, ', bag_rule[bag_rule.index('s contain ')+len('s contain'):])
            inner_bags.remove('')
            bag_rules[bag_name] = [{'bag name': inner_bag[3:], 'count': int(inner_bag[1])} for inner_bag in inner_bags]

In [ ]:
@functools.lru_cache(maxsize=len(bag_rules))
def count_inner_bags(bag_name):
    if bag_rules[bag_name]:
        return (sum(inner_bag['count']*count_inner_bags(inner_bag['bag name']) 
                   for inner_bag in bag_rules[bag_name]) + 1)
    else:
        return 1

In [ ]:
count_inner_bags('shiny gold')-1

## Day 8
### part 1

In [ ]:
instructions = pd.read_csv('8.txt', sep=' ', names=['instruction', 'value'])
visited_instructions = []
i = 0
acc = 0
while i not in visited_instructions:
    visited_instructions += [i]
    if instructions.loc[i, 'instruction'] == 'acc':
        acc += instructions.loc[i, 'value']
        i += 1
    elif instructions.loc[i, 'instruction'] == 'jmp':
        i += instructions.loc[i, 'value']
    else:
        i += 1
print(acc)

### part 2

In [ ]:
def do_instructions(instructions):
    visited_instructions = []
    i = 0
    acc = 0
    
    while (i not in visited_instructions) and (i <= instructions.index[-1]):
        visited_instructions += [i]
        if instructions.loc[i, 'instruction'] == 'acc':
            acc += instructions.loc[i, 'value']
            i += 1
            continue
        elif instructions.loc[i, 'instruction'] == 'jmp':
            i += instructions.loc[i, 'value']    
        else:
            i += 1
    return i, acc

In [ ]:
for change_index in instructions.query("instruction == 'jmp' or instruction == 'nop'").index:
    modified_instructions = instructions.copy()
    modified_instructions.loc[change_index, 'instruction'] = 'jmp' if modified_instructions.loc[change_index, 'instruction'] == 'nop' else 'nop'
    i, acc = do_instructions(modified_instructions)
    if i > modified_instructions.index[-1]:
        print(acc)
        break

## Day 9
### part 1

In [ ]:
xmas_data = np.loadtxt('9.txt', dtype=np.int64)

In [ ]:
for i, value in enumerate(xmas_data[25:]):
    combinations = np.array([combination for combination in itertools.combinations(xmas_data[i:i+25], 2)], dtype=np.int64)
    if value not in combinations.sum(axis=1):
        print(value)
        break

### part 2

In [ ]:
for low_i, high_i in itertools.combinations(range(i), 2):
    contiguous_set = xmas_data[low_i:high_i]
    if contiguous_set.sum() == value:
        print(contiguous_set.min()+contiguous_set.max())
        break

## Day 10
### part 1

In [ ]:
jolts =np.loadtxt('10.txt', dtype=np.int64)
jolts = np.append(jolts, jolts.max()+3)
jolts = np.insert(jolts, 0, 0)
jolts.sort()
jolt_diff = np.diff(jolts)
(jolt_diff == 3).sum() * (jolt_diff == 1).sum()

In [ ]:
nonmoveable = (jolt_diff == 3).nonzero()[0] + 1
all_possible_configurations = 1
for start_i, end_i in zip(np.insert(nonmoveable[:-1], 0, 0)+1, nonmoveable):
    max_int = end_i - start_i - 1
    if max_int > 0:
        possible_subsets = 0
        
        for picks in (((np.arange(2**max_int)[:,None] & (1 << np.arange(max_int)))) > 0).astype(bool).tolist():
            if (np.diff(jolts[start_i-1:end_i][np.concatenate(((True,), picks, (True,)))]) <= 3).all():
                possible_subsets += 1
        all_possible_configurations *= possible_subsets
all_possible_configurations

The code below gives the same solution as above but using Awkward Array, thus making the code vectorized. 
I was not sure how to make this efficient as the typical ak.Array(np_array) does not create a variable row length array.
I solved it by calling `extended_possible_subsets.tolist()` but I don't think this is efficient.

In [ ]:
nonmoveable = (jolt_diff == 3).nonzero()[0] + 1
all_possible_configurations = 1
for start_i, end_i in zip(np.insert(nonmoveable[:-1], 0, 0)+1, nonmoveable):
    max_int = end_i - start_i - 1
    if max_int > 0:
        possible_subsets = (((np.arange(2**max_int)[:,None] & (1 << np.arange(max_int)))) > 0).astype(bool)
        extended_possible_subsets = np.ones((2**max_int, max_int+2)).astype(bool)
        extended_possible_subsets[:, 1:-1] = possible_subsets
        allowed_subsets = ak.Array(np.tile(jolts[start_i-1:end_i], (2**max_int, 1)))[ak.Array(extended_possible_subsets.tolist())]
        allowed_subset_count = int(ak.sum(ak.all((allowed_subsets[:, 1:] - allowed_subsets[:, :-1]) <= 3, axis=1)))
        all_possible_configurations *= allowed_subset_count
all_possible_configurations

## Day 11
### part 1

In [ ]:
original_seat_state = []
with open('11.txt') as original_seat_file:
    for row in original_seat_file:
        original_seat_state.append(list(row)[:-1])

state_str_withoutpad = np.array(original_seat_state, dtype=str)
state_str = np.full(np.array(state_str.shape)+2, '.')
state_str[1:-1, 1:-1] = state_str_withoutpad
state = np.zeros(state_str.shape, dtype=bool)
seat_locs = np.array((state_str == 'L').nonzero()).T

In [ ]:
@nb.njit
def next_seat_configuration(seat_locs, state):
    new_state = state.copy()
    for index in (seat_locs):
        seats_around = state[index[0]-1:index[0]+2, index[1]-1:index[1]+2]
        if (state[index[0], index[1]] 
            and (seats_around.sum() >= 5)):
            new_state[index[0], index[1]] = False
        elif not seats_around.any():
            new_state[index[0], index[1]] = True
    return new_state

In [ ]:
while True:
    new_state = next_seat_configuration(seat_locs, state)
    if (state == new_state).all():
        print((state == True).sum())
        break
    state = new_state

### part 2
Not done yet

In [ ]:
#@nb.njit
def find_seats_around(seat_locs, state, index):
    seats_around = np.zeros((2, 8), dtype=int)
    for i, motion in enumerate(np.array([[0, 1], [0, -1], [1, 0], [1, 1], [1, -1], [-1, 0], [-1, 1], [-1, 1]], dtype=int)):
        diagonal = index + motion
        while (state_str[tuple(diagonal)] == '.') and ((0 < diagonal) & (diagonal < np.array(state.shape)-1)).all():
            diagonal += motion
        seats_around[:, i] = diagonal
    return seats_around

#@nb.njit
#def next_seat_configuration2(seat_locs, state):
#    new_state = state.copy()
#    for index in (seat_locs):
#        seats_around = find_seats_around(seat_locs, state, index)
#        if (state[index[0], index[1]] and (seats_around.sum() >= 6)):
#            new_state[index[0], index[1]] = False
#        elif not seats_around.any():
#            new_state[index[0], index[1]] = True
#    return new_state

def next_seat_configuration2(seat_locs, all_seats_around, state):
    new_state = state.copy()
    for seat, seats_around in zip(seat_locs, all_seats_around):
        if (state[tuple(seat)] and (state[seats_around[0, :], seats_around[1, :]].sum() >= 6)):
            new_state[tuple(seat)] = False
        elif not state[seats_around[0, :], seats_around[1, :]].any():
            new_state[tuple(seat)] = True
    return new_state

In [ ]:
all_seats_around = np.zeros((*seat_locs.shape, 8), dtype=int)
for i, seat in enumerate(seat_locs):
    all_seats_around[i, ...] = find_seats_around(seat_locs, state, seat)

In [ ]:
i=0
while i < 100:
    new_state = next_seat_configuration2(seat_locs, all_seats_around, state)
    if (state == new_state).all():
        print(state.sum())
        break
    state = new_state
    i += 1

In [ ]:
i

In [ ]:
seat_locs = np.array((state_str == 'L').nonzero()).T + 1
# Only allow 100 loops as it should not do require more than that. Mainly for debugging purposes
i=0
while i < 100:
    new_state = next_seat_configuration2(seat_locs, state)
    if (state == new_state).all():
        print(state.sum())
        break
    state = new_state
    i += 1

## Day 12
### part 1

In [ ]:
class Ship:
    def __init__(self, start_direction: str, start_position: complex):
        # TODO: change start_position to tuple of str
        self.direction_converter = {'N':1j, 'E':1, 'S':-1j, 'W':-1}
        self.rotation_converter = {'L':1j, 'R':-1j}
        self.direction = self.direction_converter[start_direction]
        self.start_position = start_position
        self.position = start_position
        
    def move(self, instructions: list):
        # TODO: to make this efficient, I should probably convert all instructions to numbers first
        # TODO: use reduce?
        for i in instructions:
            instruction, amount = i[0], int(i[1:])
            if instruction in self.rotation_converter.keys():
                self.direction *= self.rotation_converter[instruction]**(amount//90)
            elif instruction in self.direction_converter.keys():
                self.position += self.direction_converter[instruction]*amount
            elif instruction == 'F':
                self.position += self.direction*amount
            else:
                raise KeyError(f"invalid instruction '{instruction}'")
    
    @property
    def manhattan_distance(self):
        complex_distance = self.position - self.start_position
        return abs(complex_distance.real) + abs(complex_distance.imag)


In [ ]:
with open('12.txt') as instruction_file:
    instructions = instruction_file.read().split('\n')

In [ ]:
ferry = Ship(start_direction='E', start_position=0)
ferry.move(instructions)
ferry.manhattan_distance

### part 2

In [ ]:
class WayPointShip(Ship):
    def __init__(self, start_position: complex, start_waypoint: complex, start_direction: str = 'E'):
        # start_direction will not be used by this class
        super().__init__(start_direction, start_position)
        # TODO: change waypoint to tuple of str and convert to complex
        self.waypoint = start_waypoint
        
    def move(self, instructions: list):
        for i in instructions:
            instruction, amount = i[0], int(i[1:])
            if instruction in self.rotation_converter.keys():
                self.waypoint *= self.rotation_converter[instruction]**(amount//90)
            elif instruction in self.direction_converter.keys():
                self.waypoint += self.direction_converter[instruction]*amount
            elif instruction == 'F':
                self.position += self.waypoint*amount
            else:
                raise KeyError(f"invalid instruction '{instruction}'")

In [ ]:
waypoint_ferry = WayPointShip(start_position=0, start_waypoint=10+1j)
waypoint_ferry.move(instructions)
waypoint_ferry.manhattan_distance

## Day 13
### part 1

In [ ]:
with open('13.txt') as notes:
    time = int(notes.readline().strip('\n'))
    busses = np.array([int(bus_id) for bus_id in notes.readline().split(',')
              if bus_id != 'x'], dtype=int)

In [ ]:
bus_arrival_time = np.abs(time % busses - busses)
busses[bus_arrival_time.argmin()]* bus_arrival_time.min()

### part 2

In [ ]:
with open('13.txt') as notes:
    notes.readline()
    busses_with_shift = np.array([(int(bus_id), i%int(bus_id)) 
                       for i, bus_id in enumerate(notes.readline().split(',')) 
                       if bus_id.isnumeric()], dtype=np.int64).T

In [ ]:
busses_with_shift

In [ ]:
@nb.njit
def find_t(busses_with_shift, min_val):
    longest_bus = busses_with_shift[:, busses_with_shift[0, :].argmax()]
    min_val = 100000000000000
    (min_val + longest_bus[1]) % longest_bus[0] + min_val

    t = longest_bus[0] * ((min_val//longest_bus[0])+1) - longest_bus[1]

    while t < min_val*100:
        if ((t + busses_with_shift[1, :]) % busses_with_shift[0, :] == 0).all():
            return t
            break
        t += longest_bus[0]

In [ ]:
find_t(busses_with_shift, 100000000000000)

In [ ]:
longest_bus = busses_with_shift[:, busses_with_shift[0, :].argmax()]
min_val = 100000000000000
(min_val + longest_bus[1]) % longest_bus[0] + min_val

t = longest_bus[0] * ((min_val//longest_bus[0])+1) - longest_bus[1]

while t < np.iinfo(np.int64).max-longest_bus[0]:
    if ((t + busses_with_shift[1, :]) % busses_with_shift[0, :] == 0).all():
        print(t)
        break
    t += longest_bus[0]

In [ ]:
longest_bus = busses_with_shift[:, busses_with_shift[0, :].argmax()]
min_val = 100000000000000

((min_val + longest_bus[1]) % longest_bus[0] + min_val + longest_bus[1]) % longest_bus[0]

In [ ]:
def closestNumber(n, m) : 
    # Find the quotient 
    q = n//m
      
    # 1st possible closest number 
    n1 = m * (q+1) 
    return n1  
    

In [ ]:
t = longest_bus[0] * ((min_val//longest_bus[0])+1) -longest_bus[1]
(t + longest_bus[1]) % longest_bus[0]

## Day 14
### part 1

In [ ]:
memory = {}
int_size = 36
with open('14.txt') as bitmask_file:
    for line in bitmask_file:
        if line[:4] == 'mask':
            mask = np.array([int(i)  if i != 'X' else np.nan for i in line[7:-1]])
        elif line[:3] == 'mem':
            memory_loc = int(re.search(r'\[(.*?)\]', line).group(1))
            value = int(line[line.index(' = ')+3:-1])
            value_bin = np.array(list(np.binary_repr(value).zfill(int_size))).astype(np.int8)
            masked_value_bin = np.where(np.isnan(mask), value_bin, mask).astype(np.uint64)
            masked_value = (masked_value_bin*np.uint64(2)**np.arange(int_size-1, -1, -1).astype(np.uint64)).sum()            
            memory[memory_loc] = masked_value

In [ ]:
sum(memory.values())

### part 2

In [ ]:
memory = {}
int_size = 36
with open('14.txt') as bitmask_file:
    for line in bitmask_file:
        # This loop is quite slow and takes a few seconds
        if line[:4] == 'mask':
            mask = np.array([int(i)  if i != 'X' else np.nan for i in line[7:-1]])
        elif line[:3] == 'mem':
            adress = int(re.search(r'\[(.*?)\]', line).group(1))
            value = int(line[line.index(' = ')+3:-1])
            adress_bin = np.array(list(np.binary_repr(adress).zfill(int_size))).astype(np.int8)
            float_bit_count = np.isnan(mask).sum()
            possible_float_states = (((np.arange(2**float_bit_count)[:,None] & (1 << np.arange(float_bit_count)))) > 0)
            adress_bin[mask == 1] = 1
            for float_state in possible_float_states:
                adress_bin[np.isnan(mask)] = float_state
                masked_adress = int((adress_bin*np.uint64(2)**np.arange(int_size-1, -1, -1).astype(np.uint64)).sum())
                memory[masked_adress] = value

In [ ]:
sum(memory.values())